In [1]:
%load_ext autoreload
%autoreload 2

# Importy

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import BayesianRidge
from sklearn.neural_network import MLPRegressor
from utils import *

# Przygotowanie danych

In [3]:
dataframes = read_csv("./datasets", 'csv')

In [4]:
res = prepare_dataframes(dataframes)

  0%|          | 0/3 [00:00<?, ?it/s]

out_hum
pressure
out_temp


In [5]:
df = get_labels(res, '2024-02-13')

In [6]:
df['rain_1h'] = df['rain_1h'].fillna(0)
df['snow_1h'] = df['snow_1h'].fillna(0)
df['total percipitation'] = df['rain_1h'] + df['snow_1h']

In [7]:
df['last_changed_count'] = df.groupby('last_changed')['last_changed'].transform('count')
df = df.drop_duplicates(subset=['last_changed'])
df = df.loc[:, ['last_changed', 'pressure', 'out_temp', 'out_hum', 'total percipitation']]
df = df.reset_index(drop=True)

## Tabela 1 (time, P, T, H, R)

In [8]:
df

,last_changed,pressure,out_temp,out_hum,total percipitation
0,2023-09-20 00:00:00+00:00,986.000000,15.300000,76.000000,0.00
1,2023-09-20 01:00:00+00:00,985.428571,14.528571,78.900000,0.00
2,2023-09-20 02:00:00+00:00,985.760000,14.212500,79.500000,0.00
3,2023-09-20 03:00:00+00:00,985.280000,14.041667,81.300000,0.00
4,2023-09-20 04:00:00+00:00,985.666667,13.833333,83.333333,0.25
...,...,...,...,...,...
3499,2024-02-12 19:00:00+00:00,974.777778,7.220000,85.166667,0.00
3500,2024-02-12 20:00:00+00:00,975.846154,7.020000,84.933333,0.00
3501,2024-02-12 21:00:00+00:00,976.846154,6.566667,84.033333,0.00
3502,2024-02-12 22:00:00+00:00,976.521739,6.370000,83.873913,0.00


In [9]:
df.value_counts('total percipitation')

total percipitation
0.00     2666
0.25      166
0.51      149
0.38       37
1.02       29
         ... 
1.53        1
1.59        1
1.61        1
1.66        1
30.99       1
Name: count, Length: 119, dtype: int64

## Tabela 2 (time, 0, -1, -2, -3...)

In [10]:
df2 = generate_data(df, 5)
df2

,time,TP t+1,Press t0,Temp t0,Hum t0,TP t0,Press t-1,Temp t-1,Hum t-1,TP t-1,...,Hum t-3,TP t-3,Press t-4,Temp t-4,Hum t-4,TP t-4,Press t-5,Temp t-5,Hum t-5,TP t-5
0,2023-09-20 05:00:00+00:00,0.0,986.217391,13.966667,83.366667,0.0,985.666667,13.833333,83.333333,0.25,...,79.500000,0.00,985.428571,14.528571,78.900000,0.00,986.000000,15.300000,76.000000,0.00
1,2023-09-20 06:00:00+00:00,0.0,986.478261,14.640000,80.100000,0.0,986.217391,13.966667,83.366667,0.00,...,81.300000,0.00,985.760000,14.212500,79.500000,0.00,985.428571,14.528571,78.900000,0.00
2,2023-09-20 07:00:00+00:00,0.0,986.739130,15.480000,75.420000,0.0,986.478261,14.640000,80.100000,0.00,...,83.333333,0.25,985.280000,14.041667,81.300000,0.00,985.760000,14.212500,79.500000,0.00
3,2023-09-20 08:00:00+00:00,0.0,986.090909,17.000000,69.840000,0.0,986.739130,15.480000,75.420000,0.00,...,83.366667,0.00,985.666667,13.833333,83.333333,0.25,985.280000,14.041667,81.300000,0.00
4,2023-09-20 09:00:00+00:00,0.0,986.200000,18.600000,61.920000,0.0,986.090909,17.000000,69.840000,0.00,...,80.100000,0.00,986.217391,13.966667,83.366667,0.00,985.666667,13.833333,83.333333,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3493,2024-02-12 18:00:00+00:00,0.0,974.642857,7.371429,84.830000,0.0,973.500000,7.540000,84.800000,0.00,...,83.640000,0.00,971.652174,7.860000,85.928571,0.00,971.130435,7.955556,86.000000,0.00
3494,2024-02-12 19:00:00+00:00,0.0,974.777778,7.220000,85.166667,0.0,974.642857,7.371429,84.830000,0.00,...,83.020000,0.00,972.384615,7.877778,83.640000,0.00,971.652174,7.860000,85.928571,0.00
3495,2024-02-12 20:00:00+00:00,0.0,975.846154,7.020000,84.933333,0.0,974.777778,7.220000,85.166667,0.00,...,84.800000,0.00,973.222222,7.754545,83.020000,0.00,972.384615,7.877778,83.640000,0.00
3496,2024-02-12 21:00:00+00:00,0.0,976.846154,6.566667,84.033333,0.0,975.846154,7.020000,84.933333,0.00,...,84.830000,0.00,973.500000,7.540000,84.800000,0.00,973.222222,7.754545,83.020000,0.00


# SVR (bez normalizacji danych)

In [11]:
X = df2.iloc[:,2:-1]
y = df2.iloc[:,1:2]

In [12]:
X

,Press t0,Temp t0,Hum t0,TP t0,Press t-1,Temp t-1,Hum t-1,TP t-1,Press t-2,Temp t-2,...,Temp t-3,Hum t-3,TP t-3,Press t-4,Temp t-4,Hum t-4,TP t-4,Press t-5,Temp t-5,Hum t-5
0,986.217391,13.966667,83.366667,0.0,985.666667,13.833333,83.333333,0.25,985.280000,14.041667,...,14.212500,79.500000,0.00,985.428571,14.528571,78.900000,0.00,986.000000,15.300000,76.000000
1,986.478261,14.640000,80.100000,0.0,986.217391,13.966667,83.366667,0.00,985.666667,13.833333,...,14.041667,81.300000,0.00,985.760000,14.212500,79.500000,0.00,985.428571,14.528571,78.900000
2,986.739130,15.480000,75.420000,0.0,986.478261,14.640000,80.100000,0.00,986.217391,13.966667,...,13.833333,83.333333,0.25,985.280000,14.041667,81.300000,0.00,985.760000,14.212500,79.500000
3,986.090909,17.000000,69.840000,0.0,986.739130,15.480000,75.420000,0.00,986.478261,14.640000,...,13.966667,83.366667,0.00,985.666667,13.833333,83.333333,0.25,985.280000,14.041667,81.300000
4,986.200000,18.600000,61.920000,0.0,986.090909,17.000000,69.840000,0.00,986.739130,15.480000,...,14.640000,80.100000,0.00,986.217391,13.966667,83.366667,0.00,985.666667,13.833333,83.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3493,974.642857,7.371429,84.830000,0.0,973.500000,7.540000,84.800000,0.00,973.222222,7.754545,...,7.877778,83.640000,0.00,971.652174,7.860000,85.928571,0.00,971.130435,7.955556,86.000000
3494,974.777778,7.220000,85.166667,0.0,974.642857,7.371429,84.830000,0.00,973.500000,7.540000,...,7.754545,83.020000,0.00,972.384615,7.877778,83.640000,0.00,971.652174,7.860000,85.928571
3495,975.846154,7.020000,84.933333,0.0,974.777778,7.220000,85.166667,0.00,974.642857,7.371429,...,7.540000,84.800000,0.00,973.222222,7.754545,83.020000,0.00,972.384615,7.877778,83.640000
3496,976.846154,6.566667,84.033333,0.0,975.846154,7.020000,84.933333,0.00,974.777778,7.220000,...,7.371429,84.830000,0.00,973.500000,7.540000,84.800000,0.00,973.222222,7.754545,83.020000


In [13]:
y

,TP t+1
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
3493,0.0
3494,0.0
3495,0.0
3496,0.0


## Test 1: Poly, RBF, C=[0.1,1,10]

In [46]:
kfold = KFold(n_splits = 5)

classifiers = [    
    SVR(kernel='poly', C=1, degree=1),
    SVR(kernel='poly', C=1, degree=2),
    SVR(kernel='poly', C=1, degree=3),
    SVR(kernel='poly', C=1, degree=4), 
    
    SVR(kernel='poly', C=0.1, degree=1),
    SVR(kernel='poly', C=0.1, degree=2),
    SVR(kernel='poly', C=0.1, degree=3),
    SVR(kernel='poly', C=0.1, degree=4),
    
    SVR(kernel='poly', C=10, degree=1),
    SVR(kernel='poly', C=10, degree=2),
    SVR(kernel='poly', C=10, degree=3),
    SVR(kernel='poly', C=10, degree=4),
    
    SVR(kernel='rbf', C=0.1, degree=1),
    SVR(kernel='rbf', C=0.1, degree=2),
    SVR(kernel='rbf', C=0.1, degree=3),
    SVR(kernel='rbf', C=0.1, degree=4),
    
    SVR(kernel='rbf', C=1, degree=1),
    SVR(kernel='rbf', C=1, degree=2),
    SVR(kernel='rbf', C=1, degree=3),
    SVR(kernel='rbf', C=1, degree=4),
    
    SVR(kernel='rbf', C = 10, degree=1),
    SVR(kernel='rbf', C = 10, degree=2),
    SVR(kernel='rbf', C = 10, degree=3),
    SVR(kernel='rbf', C = 10, degree=4),   
    
    SVR(kernel='rbf', C = 20, degree=1),
    SVR(kernel='rbf', C = 20, degree=2),
    SVR(kernel='rbf', C = 20, degree=3),
    SVR(kernel='rbf', C = 20, degree=4),   
]

results_list = []
for classifier in tqdm(classifiers):
    #classifier.set_params(**{'C': 0.5})
    #print(type(classifier).__name__, classifier.get_params())

    result = {
        'Classifier': type(classifier).__name__, 
        'Kernel': classifier.get_params().get('kernel'),
        'Degree': classifier.get_params().get('degree'),
        'C': classifier.get_params().get('C'),
        'Params': classifier.get_params(),
        #**classifier.get_params(),
    }
    
    for scoring in ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']:
        score = cross_val_score(classifier, X, y.values.ravel(), cv = kfold, scoring=scoring) 
        
        result[scoring] = score.mean()

    results_list.append(result)

df_svr_1 = pd.DataFrame(results_list)

  0%|          | 0/28 [00:00<?, ?it/s]

,Classifier,Kernel,Degree,C,Params,r2,neg_mean_squared_error,neg_mean_absolute_error
0,SVR,poly,1,1.0,"{'C': 1, 'cache_size': 200, 'coef0': 0.0, 'deg...",-0.015485,-0.976886,-0.243027
1,SVR,poly,2,1.0,"{'C': 1, 'cache_size': 200, 'coef0': 0.0, 'deg...",-0.015313,-0.976824,-0.242850
2,SVR,poly,3,1.0,"{'C': 1, 'cache_size': 200, 'coef0': 0.0, 'deg...",-0.015023,-0.976656,-0.242751
3,SVR,poly,4,1.0,"{'C': 1, 'cache_size': 200, 'coef0': 0.0, 'deg...",-0.014503,-0.976414,-0.242482
4,SVR,poly,1,0.1,"{'C': 0.1, 'cache_size': 200, 'coef0': 0.0, 'd...",-0.015619,-0.976921,-0.243198
5,SVR,poly,2,0.1,"{'C': 0.1, 'cache_size': 200, 'coef0': 0.0, 'd...",-0.015630,-0.976933,-0.243131
6,SVR,poly,3,0.1,"{'C': 0.1, 'cache_size': 200, 'coef0': 0.0, 'd...",-0.015641,-0.976950,-0.243037
7,SVR,poly,4,0.1,"{'C': 0.1, 'cache_size': 200, 'coef0': 0.0, 'd...",-0.015488,-0.976911,-0.242927
8,SVR,poly,1,10.0,"{'C': 10, 'cache_size': 200, 'coef0': 0.0, 'de...",-0.014582,-0.976488,-0.242422
9,SVR,poly,2,10.0,"{'C': 10, 'cache_size': 200, 'coef0': 0.0, 'de...",-0.012745,-0.975650,-0.241366


In [ ]:
df_svr_1

## Test 2: RBF, C=[20,50,100]

In [50]:
kfold = KFold(n_splits = 5)

classifiers = [    
    SVR(kernel='rbf', C = 20, degree=1),
    SVR(kernel='rbf', C = 20, degree=2),
    SVR(kernel='rbf', C = 20, degree=3),
    SVR(kernel='rbf', C = 20, degree=4), 
    
    SVR(kernel='rbf', C = 50, degree=1),
    SVR(kernel='rbf', C = 50, degree=2),
    SVR(kernel='rbf', C = 50, degree=3),
    SVR(kernel='rbf', C = 50, degree=4),  
    
    SVR(kernel='rbf', C = 100, degree=1),
    SVR(kernel='rbf', C = 100, degree=2),
    SVR(kernel='rbf', C = 100, degree=3),
    SVR(kernel='rbf', C = 100, degree=4),  
]

results_list = []
for classifier in tqdm(classifiers):
    #classifier.set_params(**{'C': 0.5})
    #print(type(classifier).__name__, classifier.get_params())

    result = {
        'Classifier': type(classifier).__name__, 
        'Kernel': classifier.get_params().get('kernel'),
        'Degree': classifier.get_params().get('degree'),
        'C': classifier.get_params().get('C'),
        'Params': classifier.get_params(),
        #**classifier.get_params(),
    }
    
    for scoring in ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']:
        score = cross_val_score(classifier, X, y.values.ravel(), cv = kfold, scoring=scoring) 
        
        result[scoring] = score.mean()

    results_list.append(result)

df_svr_2 = pd.DataFrame(results_list)

  0%|          | 0/12 [00:00<?, ?it/s]

In [52]:
df_svr_2

,Classifier,Kernel,Degree,C,Params,r2,neg_mean_squared_error,neg_mean_absolute_error
0,SVR,rbf,1,20,"{'C': 20, 'cache_size': 200, 'coef0': 0.0, 'de...",-0.011750,-0.975101,-0.240821
1,SVR,rbf,2,20,"{'C': 20, 'cache_size': 200, 'coef0': 0.0, 'de...",-0.011750,-0.975101,-0.240821
2,SVR,rbf,3,20,"{'C': 20, 'cache_size': 200, 'coef0': 0.0, 'de...",-0.011750,-0.975101,-0.240821
3,SVR,rbf,4,20,"{'C': 20, 'cache_size': 200, 'coef0': 0.0, 'de...",-0.011750,-0.975101,-0.240821
4,SVR,rbf,1,50,"{'C': 50, 'cache_size': 200, 'coef0': 0.0, 'de...",-0.006428,-0.972473,-0.238086
5,SVR,rbf,2,50,"{'C': 50, 'cache_size': 200, 'coef0': 0.0, 'de...",-0.006428,-0.972473,-0.238086
6,SVR,rbf,3,50,"{'C': 50, 'cache_size': 200, 'coef0': 0.0, 'de...",-0.006428,-0.972473,-0.238086
7,SVR,rbf,4,50,"{'C': 50, 'cache_size': 200, 'coef0': 0.0, 'de...",-0.006428,-0.972473,-0.238086
8,SVR,rbf,1,100,"{'C': 100, 'cache_size': 200, 'coef0': 0.0, 'd...",-0.000300,-0.969174,-0.235855
9,SVR,rbf,2,100,"{'C': 100, 'cache_size': 200, 'coef0': 0.0, 'd...",-0.000300,-0.969174,-0.235855


## Test 3: RBF, C=[1e3,1e4,1e5]

In [53]:
kfold = KFold(n_splits = 5)

classifiers = [    
    SVR(kernel='rbf', C = 1000, degree=1),
    SVR(kernel='rbf', C = 1000, degree=2),
    SVR(kernel='rbf', C = 1000, degree=3),
    SVR(kernel='rbf', C = 1000, degree=4), 
    
    SVR(kernel='rbf', C = 10000, degree=1),
    SVR(kernel='rbf', C = 10000, degree=2),
    SVR(kernel='rbf', C = 10000, degree=3),
    SVR(kernel='rbf', C = 10000, degree=4),  
    
    SVR(kernel='rbf', C = 100000, degree=1),
    SVR(kernel='rbf', C = 100000, degree=2),
    SVR(kernel='rbf', C = 100000, degree=3),
    SVR(kernel='rbf', C = 100000, degree=4),  
]

results_list = []
for classifier in tqdm(classifiers):
    #classifier.set_params(**{'C': 0.5})
    #print(type(classifier).__name__, classifier.get_params())

    result = {
        'Classifier': type(classifier).__name__, 
        'Kernel': classifier.get_params().get('kernel'),
        'Degree': classifier.get_params().get('degree'),
        'C': classifier.get_params().get('C'),
        'Params': classifier.get_params(),
        #**classifier.get_params(),
    }
    
    for scoring in ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']:
        score = cross_val_score(classifier, X, y.values.ravel(), cv = kfold, scoring=scoring) 
        
        result[scoring] = score.mean()

    results_list.append(result)

df_res_rbf = pd.DataFrame(results_list) #df_svr_3

  0%|          | 0/12 [00:00<?, ?it/s]

In [54]:
df_res_rbf 

,Classifier,Kernel,Degree,C,Params,r2,neg_mean_squared_error,neg_mean_absolute_error
0,SVR,rbf,1,1000,"{'C': 1000, 'cache_size': 200, 'coef0': 0.0, '...",0.043873,-0.944373,-0.226803
1,SVR,rbf,2,1000,"{'C': 1000, 'cache_size': 200, 'coef0': 0.0, '...",0.043873,-0.944373,-0.226803
2,SVR,rbf,3,1000,"{'C': 1000, 'cache_size': 200, 'coef0': 0.0, '...",0.043873,-0.944373,-0.226803
3,SVR,rbf,4,1000,"{'C': 1000, 'cache_size': 200, 'coef0': 0.0, '...",0.043873,-0.944373,-0.226803
4,SVR,rbf,1,10000,"{'C': 10000, 'cache_size': 200, 'coef0': 0.0, ...",0.082347,-0.924145,-0.218807
5,SVR,rbf,2,10000,"{'C': 10000, 'cache_size': 200, 'coef0': 0.0, ...",0.082347,-0.924145,-0.218807
6,SVR,rbf,3,10000,"{'C': 10000, 'cache_size': 200, 'coef0': 0.0, ...",0.082347,-0.924145,-0.218807
7,SVR,rbf,4,10000,"{'C': 10000, 'cache_size': 200, 'coef0': 0.0, ...",0.082347,-0.924145,-0.218807
8,SVR,rbf,1,100000,"{'C': 100000, 'cache_size': 200, 'coef0': 0.0,...",0.074841,-0.934352,-0.203348
9,SVR,rbf,2,100000,"{'C': 100000, 'cache_size': 200, 'coef0': 0.0,...",0.074841,-0.934352,-0.203348


## Test 4: RBF, C=[1e6,1e7,1e8]

In [57]:
kfold = KFold(n_splits = 5)

classifiers = [    
    SVR(kernel='rbf', C = 1e6, degree=1),
    SVR(kernel='rbf', C = 1e6, degree=2),
    SVR(kernel='rbf', C = 1e6, degree=3),
    SVR(kernel='rbf', C = 1e6, degree=4), 
    
    SVR(kernel='rbf', C = 1e7, degree=1),
    SVR(kernel='rbf', C = 1e7, degree=2),
    SVR(kernel='rbf', C = 1e7, degree=3),
    SVR(kernel='rbf', C = 1e7, degree=4),  
    
    SVR(kernel='rbf', C = 1e8, degree=1),
    SVR(kernel='rbf', C = 1e8, degree=2),
    SVR(kernel='rbf', C = 1e8, degree=3),
    SVR(kernel='rbf', C = 1e8, degree=4),  
]

results_list = []
for classifier in tqdm(classifiers):
    #classifier.set_params(**{'C': 0.5})
    #print(type(classifier).__name__, classifier.get_params())

    result = {
        'Classifier': type(classifier).__name__, 
        'Kernel': classifier.get_params().get('kernel'),
        'Degree': classifier.get_params().get('degree'),
        'C': classifier.get_params().get('C'),
        'Params': classifier.get_params(),
        #**classifier.get_params(),
    }
    
    for scoring in ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']:
        score = cross_val_score(classifier, X, y.values.ravel(), cv = kfold, scoring=scoring) 
        
        result[scoring] = score.mean()

    results_list.append(result)

df_svr_4 = pd.DataFrame(results_list) 

  0%|          | 0/12 [00:00<?, ?it/s]

In [59]:
df_svr_4

,Classifier,Kernel,Degree,C,Params,r2,neg_mean_squared_error,neg_mean_absolute_error
0,SVR,rbf,1,1000000.0,"{'C': 1000000.0, 'cache_size': 200, 'coef0': 0...",-0.089718,-0.977693,-0.269019
1,SVR,rbf,2,1000000.0,"{'C': 1000000.0, 'cache_size': 200, 'coef0': 0...",-0.089718,-0.977693,-0.269019
2,SVR,rbf,3,1000000.0,"{'C': 1000000.0, 'cache_size': 200, 'coef0': 0...",-0.089718,-0.977693,-0.269019
3,SVR,rbf,4,1000000.0,"{'C': 1000000.0, 'cache_size': 200, 'coef0': 0...",-0.089718,-0.977693,-0.269019
4,SVR,rbf,1,10000000.0,"{'C': 10000000.0, 'cache_size': 200, 'coef0': ...",-21.276706,-5.567670,-1.587433
5,SVR,rbf,2,10000000.0,"{'C': 10000000.0, 'cache_size': 200, 'coef0': ...",-21.276706,-5.567670,-1.587433
6,SVR,rbf,3,10000000.0,"{'C': 10000000.0, 'cache_size': 200, 'coef0': ...",-21.276706,-5.567670,-1.587433
7,SVR,rbf,4,10000000.0,"{'C': 10000000.0, 'cache_size': 200, 'coef0': ...",-21.276706,-5.567670,-1.587433
8,SVR,rbf,1,100000000.0,"{'C': 100000000.0, 'cache_size': 200, 'coef0':...",-6646.395442,-1084.460670,-20.025578
9,SVR,rbf,2,100000000.0,"{'C': 100000000.0, 'cache_size': 200, 'coef0':...",-6646.395442,-1084.460670,-20.025578


In [61]:
#C=1e4 
#epsilon 0.1 domyślna najlepsza

## Test 5: RBF C=[1e4, epsilon]

In [71]:
kfold = KFold(n_splits = 5)

classifiers = [    
    SVR(kernel='rbf', C=1e4, epsilon=1e-2), 
    SVR(kernel='rbf', C=1e4, epsilon=0.05),
    SVR(kernel='rbf', C=1e4, epsilon=0.1),
    SVR(kernel='rbf', C=1e4, epsilon=0.5),
    
]

results_list = []
for classifier in tqdm(classifiers):
    #classifier.set_params(**{'C': 0.5})
    #print(type(classifier).__name__, classifier.get_params())

    result = {
        'Classifier': type(classifier).__name__, 
        'Kernel': classifier.get_params().get('kernel'),
        'Degree': classifier.get_params().get('degree'),
        'C': classifier.get_params().get('C'),
        'epsilon': classifier.get_params().get('epsilon'),
        'Params': classifier.get_params(),
        #**classifier.get_params(),
    }
    
    for scoring in ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']:
        score = cross_val_score(classifier, X, y.values.ravel(), cv = kfold, scoring=scoring) 
        
        result[scoring] = score.mean()

    results_list.append(result)

df_svr_5 = pd.DataFrame(results_list) 

  0%|          | 0/4 [00:00<?, ?it/s]

In [72]:
df_svr_5   

,Classifier,Kernel,Degree,C,epsilon,Params,r2,neg_mean_squared_error,neg_mean_absolute_error
0,SVR,rbf,3,10000.0,0.01,"{'C': 10000.0, 'cache_size': 200, 'coef0': 0.0...",0.063434,-0.939788,-0.185480
1,SVR,rbf,3,10000.0,0.05,"{'C': 10000.0, 'cache_size': 200, 'coef0': 0.0...",0.075400,-0.928601,-0.198402
2,SVR,rbf,3,10000.0,0.10,"{'C': 10000.0, 'cache_size': 200, 'coef0': 0.0...",0.082347,-0.924145,-0.218807
3,SVR,rbf,3,10000.0,0.50,"{'C': 10000.0, 'cache_size': 200, 'coef0': 0.0...",-0.414209,-1.035027,-0.501071


## Test 6: Poly, C=1e4, degree = [2,3..,9]

In [14]:
kfold = KFold(n_splits = 5)

classifiers = [    
    SVR(kernel='poly', C=1e4, degree=2),
    SVR(kernel='poly', C=1e4, degree=3), 
    SVR(kernel='poly', C=1e4, degree=4),
    SVR(kernel='poly', C=1e4, degree=5),
    SVR(kernel='poly', C=1e4, degree=6),
    SVR(kernel='poly', C=1e4, degree=7),
    SVR(kernel='poly', C=1e4, degree=8),
    SVR(kernel='poly', C=1e4, degree=9),
]

results_list = []
for classifier in tqdm(classifiers):
    #classifier.set_params(**{'C': 0.5})
    #print(type(classifier).__name__, classifier.get_params())

    result = {
        'Classifier': type(classifier).__name__, 
        'Kernel': classifier.get_params().get('kernel'),
        'Degree': classifier.get_params().get('degree'),
        'C': classifier.get_params().get('C'),
        'Params': classifier.get_params(),
        #**classifier.get_params(),
    }
    
    for scoring in ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']:
        score = cross_val_score(classifier, X, y.values.ravel(), cv = kfold, scoring=scoring) 
        
        result[scoring] = score.mean()

    results_list.append(result)

df_svr_6 = pd.DataFrame(results_list)

  0%|          | 0/8 [00:00<?, ?it/s]

In [16]:
df_svr_6 #RBF najlepsze #0.082347	-0.924145	-0.218807

,Classifier,Kernel,Degree,C,Params,r2,neg_mean_squared_error,neg_mean_absolute_error
0,SVR,poly,2,10000.0,"{'C': 10000.0, 'cache_size': 200, 'coef0': 0.0...",0.086563,-0.925648,-0.208265
1,SVR,poly,3,10000.0,"{'C': 10000.0, 'cache_size': 200, 'coef0': 0.0...",0.088438,-0.926423,-0.209572
2,SVR,poly,4,10000.0,"{'C': 10000.0, 'cache_size': 200, 'coef0': 0.0...",0.078851,-0.941661,-0.205806
3,SVR,poly,5,10000.0,"{'C': 10000.0, 'cache_size': 200, 'coef0': 0.0...",0.078474,-0.946382,-0.203363
4,SVR,poly,6,10000.0,"{'C': 10000.0, 'cache_size': 200, 'coef0': 0.0...",0.005926,-0.960528,-0.221927
5,SVR,poly,7,10000.0,"{'C': 10000.0, 'cache_size': 200, 'coef0': 0.0...",0.017567,-0.962893,-0.223925
6,SVR,poly,8,10000.0,"{'C': 10000.0, 'cache_size': 200, 'coef0': 0.0...",0.033258,-0.960124,-0.228398
7,SVR,poly,9,10000.0,"{'C': 10000.0, 'cache_size': 200, 'coef0': 0.0...",-0.075004,-0.986563,-0.249986


## Test 7: Poly, C=[1e3,1e5], degree = [1, 2, 3]

In [17]:
kfold = KFold(n_splits = 5)

classifiers = [    
    SVR(kernel='poly', C=1e3, degree=2),
    SVR(kernel='poly', C=1e3, degree=3), 
    SVR(kernel='poly', C=1e3, degree=4),
    
    SVR(kernel='poly', C=1e5, degree=2),
    SVR(kernel='poly', C=1e5, degree=3),
    SVR(kernel='poly', C=1e5, degree=4),
]

results_list = []
for classifier in tqdm(classifiers):
    #classifier.set_params(**{'C': 0.5})
    #print(type(classifier).__name__, classifier.get_params())

    result = {
        'Classifier': type(classifier).__name__, 
        'Kernel': classifier.get_params().get('kernel'),
        'Degree': classifier.get_params().get('degree'),
        'C': classifier.get_params().get('C'),
        'Params': classifier.get_params(),
        #**classifier.get_params(),
    }
    
    for scoring in ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']:
        score = cross_val_score(classifier, X, y.values.ravel(), cv = kfold, scoring=scoring) 
        
        result[scoring] = score.mean()

    results_list.append(result)

df_svr_7 = pd.DataFrame(results_list)

  0%|          | 0/6 [00:00<?, ?it/s]

In [20]:
df_svr_7 #poly C=1e4 degree 2-> 0.086563	-0.925648	-0.208265

,Classifier,Kernel,Degree,C,Params,r2,neg_mean_squared_error,neg_mean_absolute_error
0,SVR,poly,2,1000.0,"{'C': 1000.0, 'cache_size': 200, 'coef0': 0.0,...",0.055959,-0.938273,-0.224246
1,SVR,poly,3,1000.0,"{'C': 1000.0, 'cache_size': 200, 'coef0': 0.0,...",0.066383,-0.931418,-0.221799
2,SVR,poly,4,1000.0,"{'C': 1000.0, 'cache_size': 200, 'coef0': 0.0,...",0.072294,-0.928817,-0.221134
3,SVR,poly,2,100000.0,"{'C': 100000.0, 'cache_size': 200, 'coef0': 0....",0.068926,-0.933509,-0.216750
4,SVR,poly,3,100000.0,"{'C': 100000.0, 'cache_size': 200, 'coef0': 0....",0.027902,-0.956958,-0.231093
5,SVR,poly,4,100000.0,"{'C': 100000.0, 'cache_size': 200, 'coef0': 0....",-0.072857,-0.957155,-0.267308


## Grid Search

In [11]:
from sklearn.model_selection import GridSearchCV

In [12]:
X = df2.iloc[:,2:-1]
y = df2.iloc[:,1:2]

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
)

In [14]:
parameters = {
     'kernel': ['poly', 'rbf'],
     'degree': [2, 3],
     'C': [1, 10], 
     #'gamma': [1, 10],
}

In [15]:
clf = GridSearchCV(
      SVR(),
      parameters,
      scoring = 'neg_mean_squared_error',
      cv = 3)

In [18]:
clf.fit(X_train, y_train.values.ravel())

C:\Users\Wiki\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Wiki\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Wiki\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Wiki\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

GridSearchCV(cv=3, estimator=SVR(),
             param_grid={'degree': [2, 3], 'kernel': ['poly', 'rbf']},
             scoring='neg_mean_squared_error')

In [20]:
print("Najlepsze parametry:", clf.best_params_)

Najlepsze parametry: {'degree': 3, 'kernel': 'poly'}


In [21]:
clf.cv_results_

{'mean_fit_time': array([0.0800217 , 0.11509522, 0.08328525, 0.1103557 ]),
 'std_fit_time': array([0.00517555, 0.00556507, 0.00759144, 0.01210732]),
 'mean_score_time': array([0.03336795, 0.09875154, 0.03019706, 0.09592374]),
 'std_score_time': array([0.00298496, 0.00894866, 0.01149092, 0.01015769]),
 'param_degree': masked_array(data=[2, 2, 3, 3],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['poly', 'rbf', 'poly', 'rbf'],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'degree': 2, 'kernel': 'poly'},
  {'degree': 2, 'kernel': 'rbf'},
  {'degree': 3, 'kernel': 'poly'},
  {'degree': 3, 'kernel': 'rbf'}],
 'split0_test_score': array([-0.59077726, -0.5908471 , -0.59070632, -0.5908471 ]),
 'split1_test_score': array([-0.87075206, -0.87071186, -0.87067265, -0.87071186]),
 'split2_test_score': array([-1.35639129, -1.35637997, -1.3562701

In [22]:
clf.best_score_

-0.9392163595109252

In [23]:
clf.scorer_ #True?

make_scorer(mean_squared_error, greater_is_better=False)